In [2]:
!pip install findspark

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import time

In [4]:
# 1. Инициализация Spark
spark = SparkSession.builder \
    .appName("Batch_TXT_to_Parquet_Conversion") \
    .config("spark.hadoop.fs.s3a.access.key", "YOUR_ACCESS_KEY") \
    .config("spark.hadoop.fs.s3a.secret.key", "YOUR_SECRET_KEY") \
    .config("spark.hadoop.fs.s3a.endpoint", "storage.yandexcloud.net") \
    .config("spark.sql.parquet.compression.codec", "snappy") \
    .getOrCreate()


In [5]:
# 2. Схема данных (адаптирована под вашу структуру)
transaction_schema = StructType([
    StructField("transaction_id", IntegerType()),
    StructField("tx_datetime", TimestampType()),
    StructField("customer_id", IntegerType()),
    StructField("terminal_id", IntegerType()),
    StructField("tx_amount", DoubleType()),
    StructField("tx_time_seconds", IntegerType()),
    StructField("tx_time_days", IntegerType()),
    StructField("tx_fraud", IntegerType()),
    StructField("tx_fraud_scenario", IntegerType())
])


In [6]:
# 3. Пути к данным
input_bucket = "s3a://otus-mlops-data17/"
output_bucket = "s3a://fraud-detection-data-otus-2025/parquet/"

# 4. Список файлов для конвертации
files_to_convert = [
    "2019-08-22.txt",
    "2019-09-21.txt",
    "2019-10-21.txt"
]


In [7]:
# 5. Улучшенная функция конвертации с валидацией
def convert_and_validate(input_path, output_path):
    try:
        print(f"\n{'='*50}")
        print(f"Начало обработки: {input_path.split('/')[-1]}")
        start_time = time.time()
        
        # Шаг 1: Чтение CSV
        df = spark.read \
            .option("header", "false") \
            .option("delimiter", ",") \
            .schema(transaction_schema) \
            .csv(input_path)
        
        # Проверка прочитанных данных
        print("\nПроверка данных:")
        print(f"Общее количество строк: {df.count():,}")
        print("Пример данных:")
        df.show(5)
        print("Схема данных:")
        df.printSchema()
        
        # Шаг 2: Запись в Parquet
        df.write \
            .mode("overwrite") \
            .option("compression", "snappy") \
            .parquet(output_path)
        
        # Шаг 3: Валидация записи
        validation_df = spark.read.parquet(output_path)
        print("\nРезультат валидации:")
        print(f"Прочитано строк из Parquet: {validation_df.count():,}")
        validation_df.show(2)
        
        elapsed = time.time() - start_time
        print(f"\nФайл успешно конвертирован за {elapsed:.2f} секунд")
        print(f"Parquet файлы сохранены в: {output_path}")
        print(f"{'='*50}\n")
        
        return True
        
    except Exception as e:
        print(f"\nОшибка при конвертации {input_path}:")
        print(str(e))
        print(f"{'='*50}\n")
        return False


In [8]:
# 6. Пакетная конвертация трех файлов
for file_name in files_to_convert:
    input_path = f"{input_bucket}{file_name}"
    output_path = f"{output_bucket}{file_name.replace('.txt', '')}"
    
    success = convert_and_validate(input_path, output_path)
    
    if not success:
        print(f"Прерывание обработки из-за ошибки в файле {file_name}")
        break



Начало обработки: 2019-08-22.txt

Проверка данных:
Общее количество строк: 46,988,419
Пример данных:
+--------------+-------------------+-----------+-----------+---------+---------------+------------+--------+-----------------+
|transaction_id|        tx_datetime|customer_id|terminal_id|tx_amount|tx_time_seconds|tx_time_days|tx_fraud|tx_fraud_scenario|
+--------------+-------------------+-----------+-----------+---------+---------------+------------+--------+-----------------+
|          null|               null|       null|       null|     null|           null|        null|    null|             null|
|             0|2019-08-22 06:51:03|          0|        711|    70.91|          24663|           0|       0|                0|
|             1|2019-08-22 05:10:37|          0|          0|    90.55|          18637|           0|       0|                0|
|             2|2019-08-22 19:05:33|          0|        753|    35.38|          68733|           0|       0|                0|
|        

In [9]:


# 7. Финализируем сессию Spark
spark.stop()
print("Все операции конвертации  завершены")

Все операции конвертации  завершены
